In [ ]:
using Plots
using PAM

In [ ]:
import IMAS
# start with dd from PAM/ST80_case/ods
dd = IMAS.json2imas("/Users/galinaavdeeva/.julia/dev/PAM/examples/ods.txt");
# add missing (in OMFIT) time_slice to ods
dd.core_profiles.profiles_1d[1].time = dd.equilibrium.time_slice[1].time;
# save dd pellet structure for future use
old_pellets = deepcopy(dd.pellets);

In [ ]:
display(dd)

In [ ]:
# add and populate 'layer' which are needed to PAM

empty!(dd.pellets)
resize!(dd.pellets.time_slice)
resize!(dd.pellets.time_slice[].pellet, 1)
resize!(dd.pellets.time_slice[].pellet[1].layer, 1)
resize!(dd.pellets.time_slice[].pellet[1].layer[1].species, 1);
resize!(dd.pellets.time_slice[].pellet[1].layer[1].species, 2);
resize!(dd.pellets.time_slice[].pellet[1].layer, 2)
resize!(dd.pellets.time_slice[].pellet[1].layer[2].species, 1);
resize!(dd.pellets.time_slice[].pellet[1].layer[2].species, 2);
dd.pellets

In [ ]:
# modify and populta other nessesary entries
dd.pellets.time_slice[].pellet[1].layer[1].thickness = 0.2#0.05
dd.pellets.time_slice[].pellet[1].layer[2].thickness = 0.2#0.05
dd.pellets.time_slice[].pellet[1].shape = old_pellets.time_slice[].pellet[1].shape
dd.pellets.time_slice[].pellet[1].path_geometry = old_pellets.time_slice[].pellet[1].path_geometry
dd.pellets.time_slice[].pellet[1].path_profiles = old_pellets.time_slice[].pellet[1].path_profiles
dd.pellets.time_slice[].pellet[1].velocity_initial = old_pellets.time_slice[].pellet[1].velocity_initial
props = IMAS.ion_properties(:D)
props_D = IMAS.ion_properties(:T)
dd.pellets.time_slice[].pellet[1].layer[1].species[1].z_n = props.z_n
dd.pellets.time_slice[].pellet[1].layer[1].species[1].a = props.a
dd.pellets.time_slice[].pellet[1].layer[1].species[1].label = props.label

dd.pellets.time_slice[].pellet[1].layer[1].species[2].z_n = props_D.z_n
dd.pellets.time_slice[].pellet[1].layer[1].species[2].a = props_D.a
dd.pellets.time_slice[].pellet[1].layer[1].species[2].label = props_D.label


dd.pellets.time_slice[].pellet[1].layer[1].species[1].fraction = old_pellets.time_slice[].pellet[1].species[1].fraction
dd.pellets.time_slice[].pellet[1].layer[1].species[2].fraction = old_pellets.time_slice[].pellet[1].species[2].fraction


dd.pellets.time_slice[].pellet[1].layer[2].species[1].z_n = props.z_n
dd.pellets.time_slice[].pellet[1].layer[2].species[1].a = props.a
dd.pellets.time_slice[].pellet[1].layer[2].species[1].label = props.label
dd.pellets.time_slice[].pellet[1].layer[2].species[1].fraction = old_pellets.time_slice[].pellet[1].species[1].fraction


dd.pellets.time_slice[].pellet[1].layer[2].species[2].z_n = props_D.z_n
dd.pellets.time_slice[].pellet[1].layer[2].species[2].a = props_D.a
dd.pellets.time_slice[].pellet[1].layer[2].species[2].label = props_D.label
dd.pellets.time_slice[].pellet[1].layer[2].species[2].fraction = old_pellets.time_slice[].pellet[1].species[2].fraction


dd.pellets.time_slice[].pellet[1].path_geometry.second_point.z = 0.1
dd.pellets.time_slice[].pellet[1].path_geometry.second_point.phi = 0.1

radii = cumsum([layer.thickness for layer in dd.pellets.time_slice[].pellet[1].layer])
dd.pellets.time_slice[].pellet[1].shape.size[1] = 0.4#radii[end]

# ├─ a [Atomic Mass Unit]  Mass of atom
# ├─ density [atoms.m^-3]  Material density of the species in the pellet
# ├─ fraction  Atomic fraction of the species in the pellet
# ├─ label  String identifying the species (e.g. H, D, T, ...)
# ├─ sublimation_energy [eV]  Sublimation energy per atom
# └─ z_n [Elementary Charge Unit]  Nuclear charge

dd.pellets

In [ ]:
# run PAM

inputs=(
   t0 = 0,
   tf = 0.06,
   dt = 0.0001, #  time step of simulations
   debug_trajectory=true,
   debug_plasma = false
)



pellet = PAM.run_pam(dd, inputs);
#plot(pellet.time, pellet.ablation_rate;marker=:circle, markersize=1)
#display(plot!())

#plot(dd.equilibrium;cx=true)
#plot!(pellet)

In [ ]:
#plot(dd.equilibrium; cx=true)
#plot!(pellet)

plot(pellet.radius)

In [ ]:
plot(pellet.x, pellet.density_source[:,50])


In [ ]:
surfaces = IMAS.trace_surfaces(dd.equilibrium.time_slice[], IMAS.first_wall(dd.wall)...)

k=50
surface = surfaces[k]


#display(fieldnames(typeof(surface)))

#display(IMAS.flux_surface_avg(surface.Bp, surface))

#plot(surfaces, size=(2000,2000))

#plot(surface.ll, surface.Bp)
#plot(surface; marker=:circle, markersize=1)

#plot(surface.Bp)
#plot(surface.r,surface.z; marker=:circle, markersize=1, aspect_ratio=:equal)

plot([IMAS.flux_surface_avg(surface.Bp, surface) for surface in surfaces])


In [ ]:
14-6